In [3]:
env = "production"
s3_bucket = "s3://mist-aggregated-stats-{env}/".format(env=env)


# df = sqlContext.read.format('csv').options(header='true', inferSchema='true').load(s3_path)

date_day = "2020-03-0[45]"
hr = '*'

s3_capacity_path = "entity_event/entity_event-{env}/dt={day}/hr={hr}/CapacityAnomalyEvent_*.seq".format(env=env, day=date_day, hr=hr)
s3_coverage_path = "entity_event/entity_event-{env}/dt={day}/hr={hr}/CoverageAnomalyEvent_*.seq".format(env=env, day=date_day, hr=hr)


s3_capacity_path = s3_bucket + s3_capacity_path
s3_coverage_path = s3_bucket + s3_coverage_path

print(s3_capacity_path, "\n", s3_coverage_path)



user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2020-03-0[45]/hr=*/CapacityAnomalyEvent_*.seq 
 s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2020-03-0[45]/hr=*/CoverageAnomalyEvent_*.seq


# UPS site data

In [7]:
rdd_capacity = spark.sparkContext.sequenceFile(s3_capacity_path)
rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)


In [12]:
import json



rdd_capacity_USNYELE= rdd_capacity.map(lambda x: json.loads(x[1])).filter(lambda x: x.get("site_id")==user_site_USNYELE)
rdd_capacity_USFLLON= rdd_capacity.map(lambda x: json.loads(x[1])).filter(lambda x: x.get("site_id")==user_site_USFLLON)

# rdd_capacity_df = rdd_capacity.toDF()


In [13]:
rdd_capacity_USNYELE.count(), rdd_capacity_USFLLON.count()

(62, 123)

In [14]:
rdd_capacity_USNYELE.first()

{'start_time': 1583280000000,
 'end_time': 1583281200000,
 'detection_time': 1583281432884,
 'modification_time': 1583280747000,
 'occurrence': 2,
 'batch_count': 1,
 'site_id': '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1',
 'org_id': '38a18d4d-1623-4985-86d0-1bb06e5e2a48',
 'ap_id': '5c5b355297c5',
 'switch_id': None,
 'event_type': 'nonwifi',
 'event_name': 'capacity_anomaly',
 'mist_only': True,
 'enable_action': False,
 'category': 'capacity',
 'entity_id': '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1_5c5b355297c5_24',
 'entity_type': 'radio',
 'display_entity_type': 'ap',
 'detection_delay': 1432,
 'display_entity_id': '5c5b355297c5',
 'details': {'features': {'tot_anomaly_score': 4.260084810489969,
   'max_anomaly_score': 2.1300424052449847,
   'tot_anomaly_count': 2,
   'tot_nclient': 2.0840625,
   'max_nclient': 2.0,
   'channel': {'6': 2},
   'bandwidth': {'20': 2},
   'band': {'24': 2}}},
 'row_key': '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1_5c5b355297c5_24&capacity_anomaly&1583280000000',
 '

In [16]:
# df_USNYELE = rdd_capacity_USNYELE.toDF()
# df_USFLLON = rdd_capacity_USFLLON.toDF()
# rdd_capacity_USNYELE.filter(lambda x: x)

In [19]:
!pip install cassandra-driver

     |████████████████████████████████| 4.2 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 6.8 MB/s eta 0:00:01
  Created wheel for geomet: filename=geomet-0.1.2-py3-none-any.whl size=14896 sha256=0b99594a51bb6a0015f2db5baac40d835ae9f47b2cdfd7f74fe728e36f29f465
  Stored in directory: /Users/wenfengw/Library/Caches/pip/wheels/e9/eb/92/8464b4ce12209717c1728cbdc7bf3862f4a080e484701406c6
Successfully built geomet


In [26]:
from cassandra.cluster import Cluster

env = "production"

stormconf = {"cassandra.cluster":
                 ["cassandra-000-{env}.mist.pvt".format(env=env),
                                    "cassandra-001-{env}.mist.pvt".format(env=env),
                                    "cassandra-002-{env}.mist.pvt".format(env=env)
                                    ],
                        
                        
                    "cassandra.v3cluster": ["cassandra-v3-000-{env}.mist.pvt".format(env=env),
                                             "cassandra-v3-001-{env}.mist.pvt".format(env=env),
                                             "cassandra-v3-002-{env}.mist.pvt".format(env=env)
                                             ]
                 }

cassandra_host = stormconf.get('cassandra.cluster', [])
print(cassandra_host)
cluster = Cluster(cassandra_host,
                              protocol_version=3,
                              port=9042,
                              cql_version='3.2.1'
#                               connect_timeout=50.0
                              )

keyspace = "sle_anomaly"
session = cluster.connect(keyspace)

['cassandra-000-production.mist.pvt', 'cassandra-001-production.mist.pvt', 'cassandra-002-production.mist.pvt']


In [ ]:

    query = "select * from {table} where org=\'{org}\' and site=\'{site}\' and" \
            " band=\'{band}\' and when>\'{start}\' and when<\'{end}\';" \
            "".format(table=table, org=org, site=site, band=band, start=start, end=end)

    storm.log(query)
    rows = session.execute(query)

In [ ]:
df_USNYELE.show()

In [6]:
# rdd_site.take(1)
rdd_capacity_by_site= rdd_capacity.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

rdd_coverage_by_site= rdd_coverage.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 1 times, most recent failure: Lost task 1.0 in stage 6.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/rdd.p

In [3]:
user_site_USFLLON

(None,
 '{"start_time": 1583365200000, "end_time": 1583366400000, "detection_time": 1583366662251, "modification_time": 1583366247000, "occurrence": 2, "batch_count": 1, "site_id": "a799e12a-fc28-4ce7-a563-c2dd6d498e2f", "org_id": "af1e02c2-0164-4cbc-921a-96eb2a1c471a", "ap_id": "5c5b350e5267", "switch_id": null, "event_type": "nonwifi", "event_name": "capacity_anomaly", "mist_only": true, "enable_action": false, "category": "capacity", "entity_id": "a799e12a-fc28-4ce7-a563-c2dd6d498e2f_5c5b350e5267_24", "entity_type": "radio", "display_entity_type": "ap", "detection_delay": 1462, "display_entity_id": "5c5b350e5267", "details": {"features": {"tot_anomaly_score": 4.245599769099666, "max_anomaly_score": 2.122799884549833, "tot_anomaly_count": 2, "tot_nclient": 2.1120833333333335, "max_nclient": 3.0, "channel": {"1": 2}, "bandwidth": {"20": 2}, "band": {"24": 2}}}, "row_key": "a799e12a-fc28-4ce7-a563-c2dd6d498e2f_5c5b350e5267_24&capacity_anomaly&1583365200000", "radio": {"band": "24", "de

In [7]:
site_capacity_sorted = sorted(rdd_capacity_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)
site_coverage_sorted = sorted(rdd_coverage_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)


In [8]:
site_capacity_sorted[:10]


[('5cf9de1a-8034-4472-86f6-9a100ebdb792', 738),
 ('96488ea8-6dee-11e6-a43a-02e208b2d34f', 590),
 ('866808ba-1ff2-4e99-8e17-34ee1d3c70f4', 365),
 ('b3c32004-a0dc-4353-b697-1065fbd4f639', 325),
 ('64f4d304-5015-4777-911f-d71e809397c2', 315),
 ('3cf29ebc-21f1-49e5-9956-25495fe8f41b', 312),
 ('e0e7ada3-59bb-4189-8e1f-ca489812828c', 292),
 ('46247212-c9ab-47f3-ba04-ac38446640a3', 273),
 ('67151fdb-2d69-4e07-a2b9-0652c3d5bec5', 269),
 ('68fdb9ba-7b27-4293-9bf2-ad1c987c9466', 250)]

In [9]:
site_coverage_sorted[:10]

[('5cf9de1a-8034-4472-86f6-9a100ebdb792', 657),
 ('866808ba-1ff2-4e99-8e17-34ee1d3c70f4', 398),
 ('b3c32004-a0dc-4353-b697-1065fbd4f639', 357),
 ('96488ea8-6dee-11e6-a43a-02e208b2d34f', 354),
 ('a799e12a-fc28-4ce7-a563-c2dd6d498e2f', 316),
 ('3cf29ebc-21f1-49e5-9956-25495fe8f41b', 273),
 ('50fa3df4-053f-4779-86d5-f53d74343077', 225),
 ('64f4d304-5015-4777-911f-d71e809397c2', 200),
 ('469fc38e-33da-4562-8b4c-6720c43a53e0', 186),
 ('c14381b7-a3fe-4a85-ac51-11813efded19', 176)]

In [10]:
# rdd_coverage_by_site.sort()

In [16]:
rdd_capacity_by_site.get(user_site_USNYELE)

26

In [17]:
rdd_capacity_by_site.get(user_site_USFLLON)

50

In [16]:


# file_path = 's3://mist-secorapp-{env}/oc-stats-analytics/oc-stats-analytics-{env}/dt={date}/*'.format(env=env, date=DATE)

s3_apscan_bucket = 's3://mist-aggregated-stats-{ENV}/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/'.format(ENV=env)


s3_apscan_path = s3_apscan_bucket + "dt={date}/hr={hour}/last_0.25_day/*.csv".format(date=date_day, hour=hr)

print(s3_apscan_path)

# file_path = "s3a://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-02-26/hr=17/last_0.25_day/part-00000-8b45ee45-d247-4bc9-b4df-b621135297eb-c000.csv"
df_apscan= spark.read.option("header", True).csv(s3_apscan_path)

# df = spark.read.parquet(file_path)
df_apscan.printSchema()


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-03/hr=00/last_0.25_day/*.csv
root
 |-- site: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- ap2: string (nullable = true)
 |-- band: string (nullable = true)
 |-- time_epoch: string (nullable = true)
 |-- rssi: string (nullable = true)



In [17]:
from pyspark.sql.types import DateType

# df_ap.select("time_epoch").show()
df_apscan.withColumn('date', df_apscan['time_epoch'].cast(DateType())).show()


+--------------------+------------+------------+----+----------+-----+----+
|                site|          ap|         ap2|band|time_epoch| rssi|date|
+--------------------+------------+------------+----+----------+-----+----+
|00193f17-4e86-476...|5c5b357eec90|5c5b35aed787|  24|1583197064|-65.0|null|
|00193f17-4e86-476...|5c5b35aed61f|5c5b35aed791|  24|1583197062|-63.0|null|
|00193f17-4e86-476...|5c5b35aed7e6|5c5b35aed7cd|  24|1583191062|-67.0|null|
|00226d33-cd80-4b1...|5c5b35501a7c|        null|  24|1583197014|-61.0|null|
|00272816-8dc2-4f1...|5c5b357f2822|5c5b35ae6ced|  24|1583197063|-67.0|null|
|00272816-8dc2-4f1...|5c5b357f284f|5c5b35ae7152|   5|1583197062|-59.0|null|
|00272816-8dc2-4f1...|5c5b357f32c7|5c5b35ae6d1f|  24|1583196761|-76.0|null|
|00272816-8dc2-4f1...|5c5b357f3308|        null|   5|1583197064|-59.0|null|
|00272816-8dc2-4f1...|5c5b35ae6e55|5c5b35ae6c89|   5|1583197062|-64.0|null|
|00272816-8dc2-4f1...|5c5b35ae6e55|5c5b35ae6d97|   5|1583197062|-69.0|null|
|00272816-8d

In [23]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# import pyspark
# from pyspark.sql import SparkSession, Row
# from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, DataType
# import pyspark.sql.functions as fn
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
# from operator import itemgetter
# import json, datetime

In [25]:
df_apscan_site = df_apscan.filter(col("site")==("67970e46-4e12-11e6-9188-0242ac110007"))

In [29]:
df_apscan_site.count()


Py4JJavaError: An error occurred while calling o68.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 14.0 failed 1 times, most recent failure: Lost task 10.0 in stage 14.0 (TID 65, localhost, executor driver): org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-03/hr=00/last_0.25_day/part-00000-cbf445ab-42fc-42aa-b78f-c79111c394ad-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: D7299B3BAB74A4A9), S3 Extended Request ID: 6SCpTMCwGs4ZxOZ/Y1xG9ODJTwAAcK/+ZWoSILeUqQZhQI/s1ynWlgeMZCksbVXtNwWB8BvH0e0=: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: D7299B3BAB74A4A9)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateExcept

In [28]:
df_apscan_site.show()

+----+---+---+----+----------+----+
|site| ap|ap2|band|time_epoch|rssi|
+----+---+---+----+----------+----+
+----+---+---+----+----------+----+



In [ ]:
from graphframes import *